In [161]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [162]:
#import CSVs, assign headers and remove quotations from column headers
red_wine = pd.read_csv(r'C:\Users\Killian\Projects\Wine analysis\Wine data\winequality-red.csv', sep=':|;',header=0, engine='python')
red_wine.columns = red_wine.columns.str.strip('"')
white_wine = pd.read_csv(r'C:\Users\Killian\Projects\Wine analysis\Wine data\winequality-white.csv', sep=':|;',header=0, engine='python')
white_wine.columns=white_wine.columns.str.strip('"')

In [163]:
#none of these methods worked for removing quotation marks. Follow up
#red_wine[0] = red_wine[0].str.replace(r"[\"]", '')
#red_wine[0] = red_wine[0].str.replace('"','')
#red_wine.columns = red_wine.columns.str.strip('"')
#white_wine.columns = [col[1:-1] for col in white_wine.columns]


In [164]:
#checking if red wine info consistent with description
red_wine.info()
print(red_wine.shape)
print(red_wine.head(10))
print(red_wine.tail(10))
print(red_wine.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
(1599, 12)
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           7.40              0.70   

In [165]:
#checking if white wine info consistent with description
white_wine.info()
print(white_wine.shape)
print(white_wine.head(10))
print(white_wine.tail(10))
print(white_wine.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB
(4898, 12)
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           7.00              0.27   

In [166]:
# we are creating a new column called "quality_label", we define a range and associate that range with a label
red_wine['quality_label'] = red_wine['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categorical data type (specific to pandas) instead of simple string
red_wine['quality_label'] = pd.Categorical(red_wine['quality_label'],
categories=['low', 'medium', 'high'])

In [167]:
# we are creating a new column called "quality_label", we define a range and associate that range with a label
white_wine['quality_label'] = white_wine['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categorical data type (specific to pandas) instead of simple string
white_wine['quality_label'] = pd.Categorical(white_wine['quality_label'],
categories=['low', 'medium', 'high'])

In [168]:
#Create a new dataset combining red and white wines
red_wine['wine_type'] = 'red'
white_wine['wine_type'] = 'white'
wines = pd.concat([red_wine, white_wine])

# re-shuffle records just to randomize data points as you may have a problem when fitting machine learning model. Dataset may not be split in a fair way.
wines = wines.sample(frac=1, random_state=42).reset_index(drop=True)

In [181]:
#Compare descriptive stats on the red & white wine datasets.
pd.set_option('display.float_format', lambda x: '%.2f' % x) #set float to only have two decimal places
#compare each attribute separately using a simple for loop
for column in red_wine.columns:
    print(pd.concat([red_wine[[column]], white_wine[[column]]], axis=1, keys=['Red Wine','White Wine']).describe())
#used axis=1 to put them side by side. The keys allow for headings. Describe gives basics stats for comparison work.


           Red Wine    White Wine
      fixed acidity fixed acidity
count       1599.00       4898.00
mean           8.32          6.85
std            1.74          0.84
min            4.60          3.80
25%            7.10          6.30
50%            7.90          6.80
75%            9.20          7.30
max           15.90         14.20
              Red Wine       White Wine
      volatile acidity volatile acidity
count          1599.00          4898.00
mean              0.53             0.28
std               0.18             0.10
min               0.12             0.08
25%               0.39             0.21
50%               0.52             0.26
75%               0.64             0.32
max               1.58             1.10
         Red Wine  White Wine
      citric acid citric acid
count     1599.00     4898.00
mean         0.27        0.33
std          0.19        0.12
min          0.00        0.00
25%          0.09        0.27
50%          0.26        0.32
75%          0.42   